In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import subprocess
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score, balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold,RandomizedSearchCV
from sklearn.metrics import auc, roc_curve, confusion_matrix, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
data = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
#the data verified by the third person will be used as the train set finally
file =  data[['content','sentiment']]
file['sentiment'].value_counts()
label_sentiment= file['sentiment'].value_counts()
label_proportions_sentiment= file['sentiment'].value_counts(normalize=True)
print("Counts of each label_sentiment:")
print(label_sentiment)
print("\nProportions of each label_sentiment:")
print(label_proportions_sentiment)

Counts of each label_sentiment:
sentiment
Negative    262
Neither      99
Positive     48
Name: count, dtype: int64

Proportions of each label_sentiment:
sentiment
Negative    0.640587
Neither     0.242054
Positive    0.117359
Name: proportion, dtype: float64


In [3]:
import re
def remove_urls(text):
    # 定义URL的正则表达式
    url_pattern = r'https?://\S+|www\.\S+'
    # 使用re.sub()函数替换URL为空字符串
    no_url_text = re.sub(url_pattern, '', text)
    return no_url_text

def clean_text(text):
    # Remove anything that is not a letter or space
    clean_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Optional: Convert text to lower case
    clean_text = clean_text.lower()
    return clean_text

#remove stopwords
def remove_stopwords(text, stopwords):
    words = re.findall(r'\b\w+\b', text.lower())
    filtered_words = [word for word in words if word not in stopwords]
    return ' '.join(filtered_words)

dutch_stopwords = ['de', 'en', 'van', 'ik', 'te', 'dat', 
                  'die', 'in', 'een', 'hij', 'het', 'niet', 
                  'zijn', 'is', 'was', 'op', 'aan', 'met', 'als', 
                  'voor', 'had', 'er', 'maar', 'om', 'hem', 'dan', 
                  'zou', 'of', 'wat', 'mijn', 'men', 'dit', 'zo', 
                  'door', 'over', 'ze', 'zich', 'bij', 'ook', 'tot', 
                  'je', 'mij', 'uit', 'der', 'daar', 'haar', 'naar', 
                  'heb', 'hoe', 'heeft', 'hebben', 'deze', 'u', 'want', 
                  'nog', 'zal', 'me', 'zij', 'nu', 'ge', 'geen', 'omdat', 
                  'iets', 'worden', 'toch', 'al', 'waren', 'veel', 'meer', 
                  'doen', 'toen', 'moet', 'ben', 'zonder', 'kan', 'hun', 
                  'dus', 'alles', 'onder', 'ja', 'eens', 'hier', 'wie', 
                  'werd', 'altijd', 'doch', 'wordt', 'wezen', 'kunnen', 
                  'ons', 'zelf', 'tegen', 'na', 'reeds', 'wil', 'kon', 
                  'niets', 'uw', 'iemand', 'geweest', 'andere']

In [4]:
file['content'] = file['content'].apply(remove_urls) 
file['content'] = file['content'].apply(clean_text) 
file['content'] = file['content'].apply(lambda x: remove_stopwords(x, dutch_stopwords))
file['sentiment'] = file['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
y = file['sentiment']
X = file['content']

/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_1103/4109358769.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(remove_urls)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_1103/4109358769.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(clean_text)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_1103/4109358769.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [5]:
# Initialize parameters
best_score = 0
best_params = {}

#hyperparameter range
param_dist = {
    'C': np.logspace(-4, 4, 20),  # Inverse of regularization strength
    'solver': ['newton-cg', 'lbfgs', 'saga'],  # Solvers that support multinomial loss
    'max_iter': [20000, 30000,40000],  # Maximum number of iterations
}

test_scores = []
best_params_list = []
tfidf_vectorizer = TfidfVectorizer()
f1_scores = []

In [6]:
# 分层K折交叉验证
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 循环每个折叠
for train_index, test_index in outer_cv.split(X, y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainy, testy = y.iloc[train_index], y.iloc[test_index]
    
    # 使用TF-IDF变换训练数据
    tfidf_vectorizer = TfidfVectorizer()
    X_train_tfidf = tfidf_vectorizer.fit_transform(trainX)
    X_test_tfidf = tfidf_vectorizer.transform(testX)

    lr = LogisticRegression(multi_class='multinomial', random_state=42,class_weight='balanced')
    randomized_search = RandomizedSearchCV(
        estimator=lr,
        param_distributions=param_dist,
        n_iter=100,  # Number of parameter settings that are sampled
        scoring='f1_macro',
        n_jobs=-1,  # Use all available cores
        cv=inner_cv,
        random_state=42,
        verbose=1
    )

        # 拟合随机搜索
    randomized_search.fit(X_train_tfidf, trainy)

    # 获取最佳模型
    best_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_

    # 在测试集上进行预测
    test_predictions = best_model.predict(X_test_tfidf)
    test_score = f1_score(testy, test_predictions,average='macro')

    # 存储结果
    test_scores.append(test_score)
    best_params_list.append(best_params)

    print(f"Fold test score: {test_score}")
    print(f"Best parameters: {best_params}")

# 汇总结果
mean_test_score = np.mean(test_scores)
std_test_score = np.std(test_scores)

print(f"Mean test score: {mean_test_score}")
print(f"Standard deviation of test scores: {std_test_score}")
print("Best parameters for each fold:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i + 1}: {params}")

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

Fold test score: 0.3157427937915742
Best parameters: {'solver': 'newton-cg', 'max_iter': 40000, 'C': 0.00026366508987303583}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

Fold test score: 0.3633958633958634
Best parameters: {'solver': 'saga', 'max_iter': 20000, 'C': 0.03359818286283781}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


...

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

Fold test score: 0.39789377289377287
Best parameters: {'solver': 'newton-cg', 'max_iter': 40000, 'C': 0.08858667904100823}
Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The ma

Fold test score: 0.3939860652731941
Best parameters: {'solver': 'newton-cg', 'max_iter': 40000, 'C': 0.08858667904100823}
Mean test score: 0.3926499502000589
Standard deviation of test scores: 0.05782081212818474
Best parameters for each fold:
Fold 1: {'solver': 'newton-cg', 'max_iter': 40000, 'C': 0.00026366508987303583}
Fold 2: {'solver': 'saga', 'max_iter': 20000, 'C': 0.03359818286283781}
Fold 3: {'solver': 'lbfgs', 'max_iter': 20000, 'C': 0.0006951927961775605}
Fold 4: {'solver': 'newton-cg', 'max_iter': 40000, 'C': 0.08858667904100823}
Fold 5: {'solver': 'newton-cg', 'max_iter': 40000, 'C': 0.08858667904100823}


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [7]:
from collections import Counter
best_params_counter = Counter(tuple(sorted(params.items())) for params in best_params_list)
most_common_params = dict(best_params_counter.most_common(1)[0][0])

print("Most common best parameters:", most_common_params)

Most common best parameters: {'C': 0.08858667904100823, 'max_iter': 40000, 'solver': 'newton-cg'}


In [4]:
most_common_params = {'solver': 'lbfgs', 'max_iter': 20000, 'C': 0.0006951927961775605}


In [5]:
#train the entire model 
best_params =  {'solver': 'lbfgs', 'max_iter': 5000, 'C': 4.281332398719396}
file = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
file['content'] = file['content'].apply(remove_urls)
file['content'] = file['content'].apply(clean_text)
file['content'] = file['content'].apply(lambda x: remove_stopwords(x, dutch_stopwords))
file['sentiment'] = file['sentiment'].replace({'Positive': 0, 'Negative': 1, 'Neither': 2})
y = file['sentiment']
X = file['content'] 
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X)
final_model = LogisticRegression(multi_class='multinomial', random_state=42, **most_common_params,class_weight='balanced')
final_model.fit(X_train_tfidf, y)

LogisticRegression(C=0.0006951927961775605, class_weight='balanced',
                   max_iter=20000, multi_class='multinomial', random_state=42)

In [6]:
#apply the model to the test set
test = pd.read_excel('/Users/rachael/Downloads/test_df_labelled.xlsx')
test = test[['content', 'sentiment']]
test['content'] = test['content'].apply(remove_urls)
test['content'] = test['content'].apply(clean_text)
test['content'] = test['content'].apply(lambda x: remove_stopwords(x, dutch_stopwords))
test['sentiment'] = test['sentiment'].replace({'Positive': 0, 'Negative': 1, 'Neither': 2})

# Extract features and labels from test data
y_test = test['sentiment'].astype(int)
X_test = test['content']

# Transform the test data using the same TF-IDF vectorizer
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Make predictions on the test data using the trained model
test_predictions = final_model.predict(X_test_tfidf)
test_score = accuracy_score(y_test, test_predictions)

# Print test results
print(f"Test Accuracy: {test_score}")

Test Accuracy: 0.538083538083538


In [7]:
test_score = accuracy_score(y_test, test_predictions)
conf_matrix = confusion_matrix(y_test, test_predictions)
class_report = classification_report(y_test, test_predictions)
print(class_report)

              precision    recall  f1-score   support

           0       0.16      0.18      0.17        38
           1       0.65      0.67      0.66       245
           2       0.45      0.40      0.42       124

    accuracy                           0.54       407
   macro avg       0.42      0.41      0.41       407
weighted avg       0.54      0.54      0.54       407

